<h1 style="color:#333333; text-align:center; line-height: 0;"> <img style="right;" src="logo.png" width=18% height=18%> Reinforcement Learning | Assignment 2 
</h1>
<br/><br/>


The goal of this assignment is to implement:
- system 
- conditional model 
- actor 
- critic
- REINFORCE

___Total points:___ 100

###  <font color="blue"> A brief introduction </font>
Examine it carefully, it covers most of your possible needs to make an assignment.

***

### About Rcognita
The platform for this (and all subsequent work) is [Rcognita](https://gitflic.ru/project/aidynamicaction/rcognita), a framework for applying control theory and machine learning algorithms to control problems, an integral part of which is the closed-loop interaction between the agent under control and the environment evolving over time. In the Rcognita paradigm, the main bearer of all the classes and variables needed to run the simulation is the `pipeline`. 

The main parts of `pipeline` are: 
* `simulator`, which is defined at module `simulators.py` and responsible for simulation of evolution of the environment
* `actor`, defined at module `actors.py`, which is responsible for obtaining of action
* `critic`, defined at module `critics.py`, which is reponsible for learning of reward function and obtaining its value 
* `controller`, which is defined at module `controllers.py` and it's needed to put it all together into an RL (or other) controller
* `system`, which is defined at module `systems.py`.

Other minor things are also declarated in the pipeline and assembled module by module up to the execution of the pipeline itself. 
Just to be on the same page, we provide some notation to prevent further confusions.
* `weights` is the general name and for weights of neural network and for values in tables of value function and policy as well. This agreement comes from the motivation for being consistent with classical RL where critic and actor are being implemented as some neural networks with some **weights**. So, here comes the second term
* `model`. It's obvious that parameters give specificity to something. But the general form itself is being called `model`. There are plenty of models of different types and forms (such as NN). Model is what critic and actor and even running cost always have, no matter what.
* `predictor` - Inspite of it's cryptic name, this object performs an important function, namely, it carries the law by which the dynamics of our system is being predicted in future. For example, if we have some differential equation
$
\begin{cases}
\dot{\boldsymbol x} = \boldsymbol f(\boldsymbol x, \boldsymbol u)\\
\boldsymbol y = h(\boldsymbol x) \\
\boldsymbol x(0)=\boldsymbol x_{0}\\
\end{cases}
$
where $x_{0}$ is the **initial state**.
in general, there are several ways of prediction: 
> - **Analytical**, when we have a precise formula of analytical solution $\boldsymbol x(t)$ to the ODE and have no problems to compute it at any given time. This is great but not that possible in real life. Nevertheless, our predictor could be expressed like:  $\text{predictor}(\boldsymbol x(\tau),dt) = \boldsymbol x(\tau + dt)$
> - **Numerical** way is mostly a case. The simplest way of prediction then is an Euler method:
$\boldsymbol x_{k+1}= \text{predictor}(\boldsymbol x_k, \delta)=\boldsymbol x_{k}+\delta \boldsymbol f\left(\boldsymbol x_{k}, \boldsymbol u_{k}\right) \text {, }$

In this assignment we meet a new object - **scenario**. Scenario is a module that forms and executes the main loops for different scenarios, like online or episodical scenario. In this assignment we will use an episodical scenario.


<a id='Notation'></a>
### Notation summary
From now and on we will use the following notation:

| Notation &nbsp; &nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp;&nbsp; &nbsp; &nbsp; &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;| &nbsp;&nbsp;Description |
|:-----------------------:|-------------|
| $\boldsymbol f(\cdot, \cdot, \cdot) : \mathbb{R}^{n+1}\times \mathbb{R}^{m} \rightarrow \mathbb{R}^{n}$ |A **state dynamic function** or, more informally, **righ-hand-side** of a system <br /> of ordinary differential equations $\dot{\boldsymbol x} = \boldsymbol f(t, \boldsymbol x, \boldsymbol u)$|
| $\boldsymbol x \in \mathbb{R}^{n} $ | An element of the **state space** of a controlled system of dimensionality $n$ |
| $\boldsymbol u \in \mathbb{R}^{m}$ | An element of the **action space** of a controlled system of dimensionality $m$ |
| $\boldsymbol y \in \mathbb{R}^{k}$ | An **observartion**|
| $\mathbb{X}\subset \mathbb{R}^{n} $| **State constraint set**|
| $\mathbb{U}\subset \mathbb{R}^{m} $| **Action constraint set**|
| $\boldsymbol h(\cdot): \mathbb{R}^{n} \rightarrow \mathbb{R}^{k}$ | **Observation function**  |
| $\boldsymbol\rho(\cdot) : \mathbb{R}^{n} \rightarrow \mathbb{R}^{m}$ | **Policy** function |
| $r(\cdot) : \mathbb{R}^n \times \mathbb{R}^m \rightarrow \mathbb{R}$ | **Running cost** function  |


### Goal
Our main goal here is to implement the whole system almost from scratch and to apply Policy Gradient algorithm to [PID-regulator](https://en.wikipedia.org/wiki/PID_controller) coefficients (more precisely, P and D coefficient) tuning

###  <font color="blue"> Algorithm description </font>

I. **Initialization**:
- set iterations number **N_iterations**
- set episodes number **N_episodes**
- set **discount factor** $\gamma$
- initialize some **policy** parameters $\theta_0$, learning rate $\eta$

II. **Main loop**:<br/>
(Run episodical scenario)
>**for** i in range(**N_iterations**):
>>**for** j in range(**N_episodes**):
>>> **while** **time** < **t1**:
(corresponding utilized parts of Rcognita are provided in bold inside parentheses)
>>>> - simulate environment evolution (**simulator**, **system**)
>>>> - obtain observation (**system**) $\boldsymbol y_i = \boldsymbol h(\boldsymbol x_i)$
>>>> - obtain action (**actor**) $u_i \sim  \mathcal{N}(\mu,\,\sigma^{2})$
>>>> - compute and store new gradient (**actor.model**)
>>>> - update accumulated outcome (**critic**): $\sum_{i=0}^N \gamma^i \cdot\left(y_i, u_i\right)$
>>> - compute and store REINFORCE objective gradient (**scenario**): $\sum_{k=0}^N \nabla_\theta \ln \rho^\theta(u_k \vert y_k) \cdot \sum_{i=0}^N \gamma^i \cdot\left(y_i, u_i\right)$
>> - compute mean overall stored REINFORCE objective gradients (**scenario**): $\mathbb{E}\left[\sum_{k=0}^N \nabla_\theta \ln \rho^\theta(u_k \vert y_k) \cdot \sum_{i=0}^N \gamma^i \cdot\left(y_i, u_i\right)\right]$
>> - perform a gradient step (**scenario**): $\theta_{i+1}=\theta_i+\eta \mathbb{E}\left[\sum_{k=0}^N \nabla_\theta \ln \rho^\theta(u_k \vert y_k) \cdot \sum_{i=0}^N \gamma^i \cdot\left(y_i, u_i\right)\right]$

***

In [ ]:
%%capture
"""
Just importing all the necessary stuff here.
DO NOT CHANGE
"""
%matplotlib qt
%load_ext autoreload
%autoreload 2

from rcognita_framework.pipelines.pipeline_inverted_pendulum import PipelineInvertedPendulum
from rcognita_framework.rcognita.actors import ActorProbabilisticEpisodic
from rcognita_framework.rcognita.critics import CriticTrivial
from rcognita_framework.rcognita.systems import SysInvertedPendulum
from rcognita_framework.rcognita.models import ModelGaussianConditional
from rcognita_framework.rcognita.scenarios import EpisodicScenario
from rcognita_framework.rcognita.utilities import rc
import numpy as np
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import warnings

<h2 style="color:#A7BD3F;"> Section 1: System implementation </h2>

***

<img style="left;" src="n_pendulum.png" width=18% height=18%>
in our case the system has the following view
\begin{equation}
\begin{cases}
\dot{\varphi} = \theta \\
\dot{\theta} = \frac{g}{l}\sin{\varphi} + \frac{u}{ml^2} \\
\end{cases}
\end{equation}

Your task is to implement this system. More precisely, there are two crucial methods: 
* `_compute_state_dynamics(self, time, state, action, disturb)` - which computes and returns the $\boldsymbol f(t, \boldsymbol x, \boldsymbol u)$ - the right-hand-side of the system. (You should fill the `Dstate` with correct values $(\boldsymbol \varphi, \dot{\boldsymbol \varphi})$)
* `out(state, time, action)`- which yields us an observation $\boldsymbol y = \boldsymbol h(\boldsymbol x)$, where $\boldsymbol y = (\varphi, \int\limits_0^t \varphi dt, \dot{\varphi})$.

***


In [ ]:
class SysInvertedPendulumStudent(SysInvertedPendulum):
    """
    System class: mathematical inverted pendulum

    """

    def _compute_state_dynamics(self, time, state, action, disturb=[]):
        """
        Method computes state dynamics function of the system
        """

        m, g, l = self.pars[0], self.pars[1], self.pars[2]

        Dstate = ...
        #############################################
        # YOUR CODE BELOW
        #############################################

        #############################################
        # YOUR CODE ABOVE
        #############################################

        return Dstate

    def out(self, state, time=None, action=None):
        """
        Method computes observation
        """
        #############################################
        # YOUR CODE BELOW
        #############################################

        #############################################
        # YOUR CODE ABOVE
        #############################################

<h2 style="color:#A7BD3F;"> Section 2: Conditional model implementation</h2>

***

In our setting we have a stochastic policy wich is modeled by a conditional distribution $\rho^w(u | \boldsymbol y) = \frac{1}{\sqrt{\pi}}\exp{-\frac{(u-\mu)^2}{0.5}}$,
where $\boldsymbol w:=(w_1, 0, w_2)$, $\mu:=-\langle \boldsymbol w,\boldsymbol y\rangle$

Implement the following methods:
* `update_expectation(self, arg_condition)` - it should compute the expectation parameter of the distribution given the passed `arg_condition` 
* `compute_gradient(self, argin)`- self-explanatory :) Compute it yourself on the paper first.
* `update(self)` - this method is being invoked after each gradient update. So it just basically resets the model. Note that you can access to `self.arg_condition_init` for these purposes

***

`arg_condition` in this setting is a s


In [ ]:
class ModelGaussianConditionalStudent(ModelGaussianConditional):

    model_name = "model-gaussian"

    def update_expectation(self, arg_condition):
        """
        update expectation based on arg_condition
        """
        #############################################
        # YOUR CODE BELOW
        #############################################
        
        #############################################
        # YOUR CODE ABOVE
        #############################################

    def compute_gradient(self, argin):
        """
        Compute grad manually
        """
        #############################################
        # YOUR CODE BELOW
        #############################################

        #############################################
        # YOUR CODE ABOVE
        #############################################

    def update(self, new_weights):
        """
        transform the new_weights into expectation 
        and apply update_expectation method
        """
        #############################################
        # YOUR CODE BELOW
        #############################################

        #############################################
        # YOUR CODE ABOVE
        #############################################

    def sample_from_distribution(self, argin):
        self.update_expectation(argin)
        self.update_covariance()
    
        return np.array(["""here should be a normally distributed random variable 
                                            N(self.expectation, self.covariance)"""])

<h2 style="color:#A7BD3F;"> Section 3: Actor implementation</h2>

***

As you remember from the introduction, actor is responsible for the action obtaining. During the episode simulation it samples action according to it's model. By default, `update(self, observation)` method performs this operation.
But indeed we also should clip the action obtained from distribution

In [ ]:
class ActorProbabilisticEpisodicStudent(ActorProbabilisticEpisodic):

    def update(self, observation):
        """
        obtain and store the action
        """
        action_sample = ### use here sample_from_distribution from model
        self.action = np.array(
            np.clip(action_sample, self.action_bounds[0], self.action_bounds[1])
        )
        self.action_old = self.action
        current_gradient = ### compute gradient here using the corresponding model's method you've just implemented
        self.store_gradient(current_gradient)

    def update_weights_by_gradient(self, gradient, learning_rate):
        """
        Perform a step towards the gradient with some learning rate
        """
        model_weights = self.model.weights
        new_model_weights = ### A gradient step should be performed here

        self.model.update(new_model_weights)

<h2 style="color:#A7BD3F;"> Section 4: Critic implementation</h2>

***

In [ ]:
class CriticTrivialStudent(CriticTrivial):
    """
    This is a dummy to calculate outcome (accumulated running objective).
    Use an Euler method for that

    """

    def __init__(self, running_objective, sampling_time=0.01):
        self.running_objective = running_objective
        self.sampling_time = sampling_time
        self.outcome = 0

    def update_outcome(self, observation, action):
        #############################################
        # YOUR CODE BELOW
        #############################################
        self.outcome += ... ### old += new * sampling_time
        #############################################
        # YOUR CODE ABOVE
        #############################################

<h2 style="color:#A7BD3F;"> Section 4: REINFORCE</h2>

***

* Actor stores gradients in `self.actor.gradients`
* total episodic outcome can be accessed through `self.critic.outcome`

In [ ]:
class EpisodicScenarioStudent(EpisodicScenario):

    def store_REINFORCE_objective_gradient(self):
        """
        This method should compute and then append reinforce objective gradient 
        to the `self.episode_REINFORCE_objective_gradients` variable
        """
        #############################################
        # YOUR CODE BELOW
        #############################################

        #############################################
        # YOUR CODE ABOVE
        #############################################


<h2 style="color:#A7BD3F;"> Section 5: Testing</h2>

***

Here you have a full freedom of choice: you can tune whatever you want, change whatever you want. Your goal here is to beat a baseline. If you get an outcome g.t. -350, you earn 100 points. If your result lower than -700, you earn nothing.
Play with hyperparameters, choose number of episodes and number of iterations. You may also vary the length of one episode. There is plenty of work. Applying is not that straightforward, we made some necessary stuff but your main objective here is to apply your ML an mathematical intuition to obtain the best result you can. You will definitely have some questions. So do not hesitate to DM me on telegram 😊 -> @odinmaniac

Some addendums:
* The problem is pretty stochastic, so sometimes you can occasionally obtain some good results. But the point here is to achieve a convergence! So, if you didn't obtain a stabilization of parameters and loss, it doesn't count (you will be able to see it on 3-rd and 4-th subplot). So, if you think that you obtained some solid results, make plots please, or ask me to launch your notebook if you struggle with hardware or software issues
* If you're desperate, results are bad and you don't know what to do, you could try the following heuristics:
    * disable the optimization of parameter I (the second one - $\theta_2$)
    * Change learning rate (sometimes you need to change is really drammatically, so it's okay)
    * Bound your weights

In [ ]:
class PipelineInvertedPendulumStudent(PipelineInvertedPendulum):

    def initialize_system(self):
        self.system = SysInvertedPendulumStudent(
            sys_type="diff_eqn",
            dim_state=self.dim_state,
            dim_input=self.dim_input,
            dim_output=self.dim_output,
            dim_disturb=self.dim_disturb,
            pars=[self.m, self.g, self.l],
            is_dynamic_controller=self.is_dynamic_controller,
            is_disturb=self.is_disturb,
            pars_disturb=[],
        )
        self.observation_init = self.system.out(self.state_init, time=0)

    def initialize_models(self):
        super().initialize_models()
        self.actor_model = ModelGaussianConditionalStudent(
            expectation_function=self.safe_controller,
            arg_condition=self.observation_init,
            weights=self.initial_weights,
        )

    def initialize_actor_critic(self):
        self.critic = CriticTrivialStudent(
            running_objective=self.running_objective, sampling_time=self.sampling_time
        )
        self.actor = ActorProbabilisticEpisodicStudent(
            self.prediction_horizon,
            self.dim_input,
            self.dim_output,
            self.control_mode,
            self.action_bounds,
            action_init=self.action_init,
            predictor=self.predictor,
            optimizer=self.actor_optimizer,
            critic=self.critic,
            running_objective=self.running_objective,
            model=self.actor_model,
        )

    def initialize_scenario(self):
        self.scenario = EpisodicScenarioStudent(
            system=self.system,
            simulator=self.simulator,
            controller=self.controller,
            actor=self.actor,
            critic=self.critic,
            logger=self.logger,
            datafiles=self.datafiles,
            time_final=self.time_final,
            running_objective=self.running_objective,
            no_print=self.no_print,
            is_log=self.is_log,
            is_playback=self.is_playback,
            N_episodes=self.N_episodes,
            N_iterations=self.N_iterations,
            state_init=self.state_init,
            action_init=self.action_init,
        )

    def execute_pipeline(self, **kwargs):
        """
        Full execution routine
        """
        np.random.seed(42)
        self.load_config()
        self.setup_env()
        self.__dict__.update(kwargs)
        self.initialize_system()
        self.initialize_predictor()
        self.initialize_safe_controller()
        self.initialize_models()
        self.initialize_objectives()
        self.initialize_optimizers()
        self.initialize_actor_critic()
        self.initialize_controller()
        self.initialize_simulator()
        self.initialize_logger()
        self.initialize_scenario()
        if not self.no_visual and not self.save_trajectory:
            self.initialize_visualizer()
            self.main_loop_visual()
        else:
            self.scenario.run()
            if self.is_playback:
                self.playback()
                
    #def playback(self):
    #    self.initialize_visualizer()
    #    anm = animation.FuncAnimation(
    #        self.animator.fig_sim,
    #        self.animator.playback,
    #        init_func=self.animator.init_anim,
    #        blit=False,
    #        interval=self.sampling_time / 1e6,
    #        repeat=False,
    #    )
#
    #    self.animator.get_anm(anm)
    #    self.animator.speedup = self.speedup
#
    #    cId = self.animator.fig_sim.canvas.mpl_connect(
    #        "key_press_event", lambda event: on_key_press(event, anm)
    #    )
#
    #    anm.running = True
#
    #    self.animator.fig_sim.tight_layout()
    #    plt.show()
    
##### Execution here!!! Full list of kwargs can be seen at 
##### rcognita_framework.pipelines.config_blueprints in the ConfigInvertedPendulum
pipeline = PipelineInvertedPendulumStudent()
pipeline.execute_pipeline(
    no_visual=True, 
    time_final=10, 
    speedup=50,
    is_playback=True, 
    N_episodes=3, 
    N_iterations=8, 
    learning_rate=0.01,
    initial_weights=[1., 0., 1.],
    sampling_time=0.1, # Do not change it!
    no_print=True)

+---------------------+-------------------+----------+---------------+---------------------+-----------+---------------------+-----------+
|   iterations_passed |   episodes_passed |    t [s] |   angle [rad] |   angle_dot [rad/s] |   M [N m] |   running_objective |   outcome |
+=====================+===================+==========+===============+=====================+===========+=====================+===========+
|                   1 |                 1 |    7.929 |         3.088 |              -0.106 |     0.814 |             -96.052 |  -741.883 |
+---------------------+-------------------+----------+---------------+---------------------+-----------+---------------------+-----------+
+---------------------+-------------------+----------+---------------+---------------------+-----------+---------------------+-----------+
|   iterations_passed |   episodes_passed |    t [s] |   angle [rad] |   angle_dot [rad/s] |   M [N m] |   running_objective |   outcome |
+=====================+====

+---------------------+-------------------+----------+---------------+---------------------+-----------+---------------------+-----------+
|   iterations_passed |   episodes_passed |    t [s] |   angle [rad] |   angle_dot [rad/s] |   M [N m] |   running_objective |   outcome |
+=====================+===================+==========+===============+=====================+===========+=====================+===========+
|                   1 |                 2 |    3.919 |         2.964 |               0.106 |     0.740 |             -88.445 |  -340.475 |
+---------------------+-------------------+----------+---------------+---------------------+-----------+---------------------+-----------+
+---------------------+-------------------+----------+---------------+---------------------+-----------+---------------------+-----------+
|   iterations_passed |   episodes_passed |    t [s] |   angle [rad] |   angle_dot [rad/s] |   M [N m] |   running_objective |   outcome |
+=====================+====

+---------------------+-------------------+----------+---------------+---------------------+-----------+---------------------+-----------+
|   iterations_passed |   episodes_passed |    t [s] |   angle [rad] |   angle_dot [rad/s] |   M [N m] |   running_objective |   outcome |
+=====================+===================+==========+===============+=====================+===========+=====================+===========+
|                   1 |                 2 |    9.919 |         2.957 |              -0.170 |     0.289 |             -87.594 |  -935.035 |
+---------------------+-------------------+----------+---------------+---------------------+-----------+---------------------+-----------+
+---------------------+-------------------+----------+---------------+---------------------+-----------+---------------------+-----------+
|   iterations_passed |   episodes_passed |    t [s] |   angle [rad] |   angle_dot [rad/s] |   M [N m] |   running_objective |   outcome |
+=====================+====

+---------------------+-------------------+----------+---------------+---------------------+-----------+---------------------+-----------+
|   iterations_passed |   episodes_passed |    t [s] |   angle [rad] |   angle_dot [rad/s] |   M [N m] |   running_objective |   outcome |
+=====================+===================+==========+===============+=====================+===========+=====================+===========+
|                   2 |                 0 |    5.907 |         3.168 |               0.008 |    -0.461 |            -100.603 |  -536.317 |
+---------------------+-------------------+----------+---------------+---------------------+-----------+---------------------+-----------+
+---------------------+-------------------+----------+---------------+---------------------+-----------+---------------------+-----------+
|   iterations_passed |   episodes_passed |    t [s] |   angle [rad] |   angle_dot [rad/s] |   M [N m] |   running_objective |   outcome |
+=====================+====

### Grading!

In [ ]:
pipeline = PipelineInvertedPendulumStudent()
pipeline.execute_pipeline(
    no_visual=True, 
    t1=10, 
    is_playback=False, 
    N_episodes= , ##### set your episodes number 
    N_iterations=1,##### set your iterations number 
    initial_weights=[1., 0., 1.],
    no_print=True)

mean_episodic = pipeline.scenario.outcome_episodic_means[0]
grade = np.clip(0.28 * mean_episodic + 200, 0, 100)
print(f"Your grade: {grade}")

In [ ]:
mean_episodic